In [6]:
from pymongo import MongoClient, UpdateOne
import pymongo
import gender_guesser.detector as gender

gender_detector = gender.Detector(case_sensitive=False)

client = MongoClient("localhost", 27017)
db = client.final_orcid_database

In [7]:
def gender_from_name(given_names, gender_api):
    given_names_list = given_names.split()
    final_guess = "unknown"

    for given_name in given_names_list:
        gender_guess = gender_api(given_name)
        if gender_guess == "unknown":
            continue
        elif gender_guess == "andy":
            return final_guess
        else:
            return gender_guess

    return final_guess

In [ ]:
all_documents = db.authors.find()

bulk_updates = []
counter = 0
for document in all_documents:
    if "given_names" in document:
        given_names = document["given_names"]
        guessed_gender = gender_from_name(given_names, gender_detector.get_gender)
        bulk_updates.append(
            UpdateOne( { "_id": document["_id"] }, { "$set": { "gender": guessed_gender } })
        )
    else:
        bulk_updates.append(
            UpdateOne( { "_id": document["_id"] }, { "$set": { "gender": "unknown" } })
        )

    counter += 1
    if counter % 10000 == 0:
        print(counter)
        db.authors.bulk_write(bulk_updates)
        bulk_updates = []

db.authors.bulk_write(bulk_updates)

print(counter)